In [ ]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions, HoverTool, ColorBar, LinearColorMapper
from bokeh.palettes import all_palettes, Plasma5
from bokeh.plotting import gmap
from bokeh.transform import linear_cmap
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('cu-data/wifi_df.csv')
# csv file with the following columns: 'Timestamp', 'Lat', 'Lon', 'Download_Speed', 'Download_Size', 'Upload_Speed', 'Upload_Size', 'Latency', 'Server', 'InternalIp', 'ExternalIp', 'URL', 'Signal_Strength'
# the important ones are 'Timestamp', 'Lat', 'Lon', 'Download_Speed', 'Upload_Speed', 'Signal_Strength'
print(df) 

In [ ]:
output_file("gmap.html")

In [ ]:
map_options = GMapOptions(lat=40.104821, lng=-88.227228, map_type="roadmap", zoom=15)

In [ ]:
# use this api key: https://developers.google.com/maps/documentation/javascript/overview
google_maps_javascript_api_key = ""
p = gmap(google_maps_javascript_api_key, map_options, title="Champaign-Urbana Campustown")

In [ ]:
factors = df['Signal_Strength'].unique().tolist()
len(factors)

In [ ]:
source = ColumnDataSource(data=df)

# color_mapper = linear_cmap(field_name = 'Download_Speed', palette = Plasma5, low = df['Download_Speed'].min(), high = df['Download_Speed'].max())
# tooltips = [("WiFi Signal Strength (dBm)","@Signal_Strength"), ("Download Speed (Mbps)","@Download_Speed"),("Upload Speed (Mbps)","@Upload_Speed")]
color_mapper = linear_cmap(field_name = 'Signal_Strength', palette = Plasma5, low = df['Signal_Strength'].min(), high = df['Signal_Strength'].max())
tooltips = [("5G Signal Strength (dBm)","@Signal_Strength")]

p.add_tools(HoverTool(tooltips=tooltips))

p.circle(x="Lon", y="Lat", color = color_mapper, fill_alpha=0.6, size = 25, source=source)
color_bar = ColorBar(color_mapper=color_mapper['transform'], label_standoff = 10, border_line_color=None, location=(0,0))
p.add_layout(color_bar, 'right')

In [ ]:
show(p)